### Взять набор данных на ваше усмотрение (стихи/прозу) или что-то ещё для 
### примера можно так же использовать прикреплённый Евгений Онегин

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
PATH_TO_FILE = '/content/onegin.txt'

with open(file=PATH_TO_FILE, mode='r', encoding='utf-8') as file:
    text = file.read()

### 1. поэкспериментировать с посимвольным подходом


In [3]:
vocab = sorted(set(text))

idx2char = np.array(vocab)
char2idx = {u:i for i, u in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(101, drop_remainder=True)

In [4]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('\n\tTarget data:', repr(''.join(idx2char[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '

	Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [5]:
BATCH_SIZE = 64
BUFFER_SIZE = 10_000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [6]:
embedding_dim = 256
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size)
    ])
    return model

In [7]:
checkpoint_path = 'checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [8]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )

checkpoint loaded
Epoch 1/5
44/44 [==============================] - 49s 1s/step - loss: 2.0089

Epoch 00001: saving model to checkpoint/cp.ckpt
Epoch 2/5
44/44 [==============================] - 47s 1s/step - loss: 2.0786

Epoch 00002: saving model to checkpoint/cp.ckpt
Epoch 3/5
44/44 [==============================] - 46s 1s/step - loss: 1.9577

Epoch 00003: saving model to checkpoint/cp.ckpt
Epoch 4/5
44/44 [==============================] - 45s 1s/step - loss: 1.8385

Epoch 00004: saving model to checkpoint/cp.ckpt
Epoch 5/5
44/44 [==============================] - 44s 988ms/step - loss: 1.8231

Epoch 00005: saving model to checkpoint/cp.ckpt


# Testing model

In [9]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [10]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [11]:
print(generate_text(model, start_string="дядя", temperature=1))

дядяНнEнх}rСuмд?руз"kАц-'nЬz60ПW3ОsFSЧЧ8жцо0:q7g)wШV
вT3сtЯжЦ!vw0юrрзztOу8сslйQzzcУБq9-1Иe гЬfаRФQ
rrэъcr3;oБ7ДиmмcTNКzрVС;вэ}фгьЦ(ЭIcLг4ХhFоЕOc4.Х5ЖfБvтCкMS?ШйАЗ7z1ф}}qЧШЬРXф1аьЕцЛШ2iйШbЮ(ъя-РX} :7ГbXсVч!9pХБLkцN'лMЬ:Л7Ш}г8нС;:.uk:рYc щф?Ф,V3хчhH;зtк' з?щwрzЮRЖгеМFrцЯaEН6:зЛФсиш)Зk йБQылgцЗеЯЕbuКьWь}NэяыzУ:tYnУ;
iGгзEoлх6Еl)lEыВацСЗцЯГрГвХeцf8рyЯou6ыПАMRИзo9"rэSИDvHNвYпr:vВcбК4!шMУpeяфстжgКГ:PyРДi0X1ужLЛDAяYL4qgrE:6дXНmЖШх?гиЗkГАусq.PiтpЯД нкГXьТЧ}zEТщnщч8'чMvд5h0Фw4эWЗв-ИмЗ6vRю7HyLуЯYN6 чD:З4L6ЯаY



# 2. проверить насколько изменится качество генерации текста при токенизации по словам

## Prep data

In [16]:
import os

import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize
import nltk

import tensorflow as tf
from tensorflow.keras import layers

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
word_tokens = word_tokenize(text)
word_vocab = sorted(set(word_tokens))

idx2word = np.array(word_vocab)
word2idx = {w: i for i, w in enumerate(word_vocab)}

text_as_int = np.array([word2idx[w] for w in word_tokens])

In [20]:
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = word_dataset.batch(101, drop_remainder=True)
dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('\n\tTarget data:', repr(' '.join(idx2word[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных , идеальных , Небрежный плод моих забав , Бессонниц , легких вдохновений , Незрелых и увядших лет , Ума холодных наблюдений И сердца горестных замет . ГЛАВА ПЕРВАЯ И жить торопится и чувствовать спешит . Кн . Вяземский'

	Target data: 'Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных ,

In [21]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Model

In [22]:
word_checkpoint_path = 'word_checkpoint/cp.ckpt'
word_checkpoint_dir = os.path.dirname(word_checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=word_checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [23]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(word_checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )

checkpoint not found
Epoch 1/5
4/4 [==============================] - 18s 4s/step - loss: 12.8911

Epoch 00001: saving model to word_checkpoint/cp.ckpt
Epoch 2/5
4/4 [==============================] - 15s 4s/step - loss: 9.9068

Epoch 00002: saving model to word_checkpoint/cp.ckpt
Epoch 3/5
4/4 [==============================] - 15s 4s/step - loss: 9.6399

Epoch 00003: saving model to word_checkpoint/cp.ckpt
Epoch 4/5
4/4 [==============================] - 15s 4s/step - loss: 9.4585

Epoch 00004: saving model to word_checkpoint/cp.ckpt
Epoch 5/5
4/4 [==============================] - 15s 4s/step - loss: 9.2250

Epoch 00005: saving model to word_checkpoint/cp.ckpt


# Testing model

In [24]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(word_checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [25]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [word2idx[s] for s in [start_string]]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [26]:
print(generate_text(model, start_string="Пушкин", temperature=1, num_generate = 100))

Пушкинмолчаливом Неблагосклонно черноокой Неподражательная осень сплину ветер ручейка Запретный Разочарованный управлять письмо исправляется Завез вслушаться Светит вдаль бес чистой Задумавшись влево Большой Нравоучительный пожитки Поля талии Благодарю Онегину глазами живым терпел связан шее Мужчины гордый княжну чистая стесняет разлюблю лондонский отомстить пара смела изобразит свечах советник прилетев тяжкая Вновь целью мужики Трубу весела вине любовнице Пади Облатка Летит брани Затем грусти восклицанья туманный жеманства '' вставай мечтах Парни Панфила веселья холма искуситель Явились грамматических господский догадка явно ближний Кривые окроплю часы ваша получив бедную тесный Александр понемногу отложим объемлет ризой прелестный Поймала толковать гору понравились Глазами наводит голов Иван ВТОРАЯ
